# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [54]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F 
import json

from PIL import Image
from torch import nn, optim
from torchvision import datasets, transforms, models

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [127]:
# TODO: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomRotation(90),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.CenterCrop(224),
                                      transforms.Resize(255),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [ ]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

# TODO: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform = train_transforms)                          
valid_data = datasets.ImageFolder(valid_dir, transform = valid_transforms)
test_data = datasets.ImageFolder(test_dir, transform = test_transforms)

In [141]:
print(train_data.class_to_idx)

{'1': 0, '10': 1, '100': 2, '101': 3, '102': 4, '11': 5, '12': 6, '13': 7, '14': 8, '15': 9, '16': 10, '17': 11, '18': 12, '19': 13, '2': 14, '20': 15, '21': 16, '22': 17, '23': 18, '24': 19, '25': 20, '26': 21, '27': 22, '28': 23, '29': 24, '3': 25, '30': 26, '31': 27, '32': 28, '33': 29, '34': 30, '35': 31, '36': 32, '37': 33, '38': 34, '39': 35, '4': 36, '40': 37, '41': 38, '42': 39, '43': 40, '44': 41, '45': 42, '46': 43, '47': 44, '48': 45, '49': 46, '5': 47, '50': 48, '51': 49, '52': 50, '53': 51, '54': 52, '55': 53, '56': 54, '57': 55, '58': 56, '59': 57, '6': 58, '60': 59, '61': 60, '62': 61, '63': 62, '64': 63, '65': 64, '66': 65, '67': 66, '68': 67, '69': 68, '7': 69, '70': 70, '71': 71, '72': 72, '73': 73, '74': 74, '75': 75, '76': 76, '77': 77, '78': 78, '79': 79, '8': 80, '80': 81, '81': 82, '82': 83, '83': 84, '84': 85, '85': 86, '86': 87, '87': 88, '88': 89, '89': 90, '9': 91, '90': 92, '91': 93, '92': 94, '93': 95, '94': 96, '95': 97, '96': 98, '97': 99, '98': 100, '99'

In [146]:
# Ensure consistent indexing

class_to_idx = train_data.class_to_idx

idx_to_class = {v: k for k, v in class_to_idx.items()}

valid_data.class_to_idx = class_to_idx
test_data.class_to_idx = class_to_idx

# test_data.class_to_idx
# [i for i in enumerate(test_data.samples)]

valid_data.samples = [(path, class_to_idx[path.split('/')[-2]]) for path, _ in valid_data.samples]
test_data.samples = [(path, class_to_idx[path.split('/')[-2]]) for path, _ in test_data.samples]

# test_data.samples = [(path, class_to_idx[class_name]) for path, class_name in test_data.samples]

# valid_data.samples
test_data.samples
# [i for i in valid_data.samples]

[('flowers/test/1/image_06743.jpg', 0),
 ('flowers/test/1/image_06752.jpg', 0),
 ('flowers/test/1/image_06754.jpg', 0),
 ('flowers/test/1/image_06760.jpg', 0),
 ('flowers/test/1/image_06764.jpg', 0),
 ('flowers/test/10/image_07090.jpg', 1),
 ('flowers/test/10/image_07104.jpg', 1),
 ('flowers/test/10/image_07117.jpg', 1),
 ('flowers/test/100/image_07896.jpg', 2),
 ('flowers/test/100/image_07897.jpg', 2),
 ('flowers/test/100/image_07899.jpg', 2),
 ('flowers/test/100/image_07902.jpg', 2),
 ('flowers/test/100/image_07926.jpg', 2),
 ('flowers/test/100/image_07936.jpg', 2),
 ('flowers/test/100/image_07938.jpg', 2),
 ('flowers/test/100/image_07939.jpg', 2),
 ('flowers/test/101/image_07949.jpg', 3),
 ('flowers/test/101/image_07952.jpg', 3),
 ('flowers/test/101/image_07983.jpg', 3),
 ('flowers/test/101/image_07988.jpg', 3),
 ('flowers/test/102/image_08004.jpg', 4),
 ('flowers/test/102/image_08012.jpg', 4),
 ('flowers/test/102/image_08015.jpg', 4),
 ('flowers/test/102/image_08023.jpg', 4),
 ('fl

In [33]:
# TODO: Using the image datasets and the trainforms, define the dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size = 40, shuffle = True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size = 40)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = 40)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [34]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

## Note for Workspace users: 
If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [3]:
# TODO: Build and train your network
# import json
# with open('cat_to_name.json', 'r') as f:
#     cat_to_name = json.load(f)

model = models.vgg16(pretrained = True)
model

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:06<00:00, 79082607.42it/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [7]:
layer_size = model.classifier[0].in_features
layer_1 = layer_size//16
layer_2 = layer_size//32
n_classes = len(cat_to_name)
print(layer_size, layer_1, layer_2, n_classes)

for param in model.parameters():
    param.requires_grad = False
    
classifier = nn.Sequential(nn.Linear(layer_size, layer_1),
                          nn.ReLU(), #capitalization important for ReLU in torch
                          nn.Dropout(p = 0.2),
                          nn.Linear(layer_1, layer_2),
                          nn.ReLU(), #capitalization important for ReLU in torch
                          nn.Dropout(p = 0.2), 
                          nn.LogSoftmax(dim = 1))
model.classifier = classifier

criterion = nn.NLLLoss()

optimizer = optim.Adam(model.classifier.parameters(), lr = 0.003)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device

model.to(device);

25088 1568 784 102


In [9]:
###########################################################

#Trains model

epochs = 1
steps = 0
print_every = 40


for n in range(epochs):
    running_loss = 0
    for inputs, labels in train_loader: #labels are flower names, need to use "flowers" in next loop
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logpred = model.forward(inputs)
        loss = criterion(logpred, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps%print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in valid_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logpred = model(inputs)
                    batch_loss = criterion(logpred, labels)
                    
                    test_loss += batch_loss.item() #this is tested against the valid data set
                    
                    ps = torch.exp(logpred)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                training_loss = running_loss/print_every    
                avg_validation_loss = test_loss / len(valid_loader)
                avg_validation_accuracy = accuracy / len(valid_loader)
                    
            print("Epoch: {}/{}  ".format(n+1, epochs),
                  "Training Loss: {:.3f}  ".format(training_loss),
                  "Validation Loss: {:.3f}  ".format(avg_validation_loss),
                  "Validation Accuracy: {:.3f}".format(avg_validation_accuracy))
            running_loss = 0
            model.train()

Epoch: 1/1   Training Loss: 7.239   Validation Loss: 6.664   Validation Accuracy: 0.010
Epoch: 1/1   Training Loss: 6.664   Validation Loss: 6.664   Validation Accuracy: 0.010
Epoch: 1/1   Training Loss: 6.664   Validation Loss: 6.664   Validation Accuracy: 0.010
Epoch: 1/1   Training Loss: 6.664   Validation Loss: 6.664   Validation Accuracy: 0.010


## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [10]:
#Tests model against the test data and gives model accuracy
            
correct = 0
total = 0
with torch.no_grad():
    model.eval()
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print(100 * (correct/total))

0.6105006105006106


In [96]:
print(model.classifier)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)


In [24]:
# TODO: Save the checkpoint 

model.class_to_idx = train_data.class_to_idx
model_state = {'state_dict': model.state_dict(),
               'classifier': model.classifier,
               'class_to_idx': model.class_to_idx,
              }

torch.save(model_state, 'checkpoint.pth')

NameError: name 'train_data' is not defined

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [25]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = models.vgg13(pretrained = True)
    model.state_dict = checkpoint['state_dict']
    # make sure model.classifier exists?
    model.classifier = checkpoint['classifer']
    # model.classifier = checkpoint['classifier']    # it should be this fix me later
    model.class_to_idx = checkpoint['class_to_idx']
    
    return model

saved_model = load_checkpoint('checkpoint.pth')
saved_model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16): ReLU(inplace)
    (17): Conv2d

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [58]:
from PIL import Image

def process_image(image_path):
    """Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    """

    mean = [0.485, 0.456, 0.406]
    stdev = [0.229, 0.224, 0.225]

    img = Image.open(image_path)
    img = transforms.CenterCrop(224)(img)
    img = transforms.Resize(255)(img)

    np_img = np.array(img) / 255
    np_img = (np_img - mean) / stdev
    np_img = np_img.transpose(2, 0, 1)

    return np_img

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [27]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax


#img = process_image("data/train/2/image_05087.jpg")
#print("After resizing, cropping and normalizing, size: {}".format(img.shape))
#imshow(img)



## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [147]:
image_path = test_dir + '/102/image_08030.jpg'

image = torch.from_numpy(np.expand_dims(process_image(image_path),axis=0)).type(torch.FloatTensor)

if device=='cuda':
    saved_model.cuda()
else:
    saved_model.cpu()

saved_model = saved_model.eval()

if device=='cuda':
    image = image.cuda()

with torch.no_grad():
    output = saved_model(image)

prob = torch.exp(output)
top_p, top_class_idx = prob.topk(5, dim=1)

top_class_folders = [[idx_to_class[idx.item()] for idx in row] for row in top_class_idx]





# Reverse dict to map idx to class name
# idx_to_class = {val: key for key, val in test_data.class_to_idx.items()}

# Convert top class indices to folder names
# top_class_folders = [idx_to_class[idx.item()] for idx in top_class_idx[0]]

# print(top_class_folders)
# mapped_classes = []

# for label in top_class.numpy()[0]:
#     mapped_classes.append(idx_to_class[label])

# print(top_class.numpy())
# print(top_class.numpy()[0])


# return top_p, mapped_classes

KeyError: 258

In [126]:
for idx in top_class_idx[0]:
    if idx.item() not in idx_to_class:
        print("Missing index:", idx.item())
        


Missing index: 258
Missing index: 261
Missing index: 260
Missing index: 259
Missing index: 783


In [102]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    image = torch.from_numpy(np.expand_dims(process_image(image_path),axis=0)).type(torch.FloatTensor)
    
    # TODO: Implement the code to predict the class from an image file
    if device=='cuda':
        model.cuda()
    else:
        model.cpu()
       
    model = model.eval()
    
    if device=='cuda':
        image = image.cuda()
    with torch.no_grad():
        output = model(image)
    
#     output = model.forward(image)
    
    prob = torch.exp(output)
    top_p, top_class = prob.topk(topk)   
    
    [c for c in top_class]
    
#     if device == 'cuda':
#         model.cuda()
#     else:
#         model.cpu()
#     model = model.eval()
    
#     if device == 'cuda':
#         image = image.cuda()
#     with torch.no_grad():
#         output = model(image)
        
#         prob = torch.exp(output)
#         top_p, top_class = prob.topk(topk)
        
#     idx_to_class = {val: key for key, val in model.class_to_idx}
    idx_to_class = {val: key for key, val in model.class_to_idx.items()}
    
    mapped_classes = []
    
#     for label in top_class.numpy()[0]:
#         mapped_classes.append(idx_to_class[label])
    
    print(top_class.numpy())
    print(top_class.numpy()[0])
    
    
    return top_p, mapped_classes

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [76]:
# TODO: Display an image along with the top 5 classes
from PIL import Image

image_path = test_dir + '/102/image_08030.jpg'
print(image_path)

img_num = image_path.split('/')[2]

print(img_num)

flower = cat_to_name[img_num]

print(flower)

img = Image.open(image_path)

print(img)


probs, names = predict(image_path, model=saved_model)
# label = names[0]

# fig = plt.figure(figsize = (10, 10))
# # plt.show()
# sp_img = plt.subplot2grid((15,9), (0,0), colspan=9, rowspan=9)
# sp_prd = plt.subplot2grid((15,8), (9,2), colspan=5, rowspan=5)
# sp_img.set_title(f'{flower}')

# disp = Image.open(image_path)
# sp_img.set_title(f'{flower}')
# sp_img.imshow(disp)

# labels = []
# for class_idx in names:
#     labels.append(cat_to_name[class_idx])
# yp = np.arange(5)
# sp_prd.set_yticklabels(labels)
# sp_prd.barh(yp, align = 'center', color = 'blue')

# plt.show()


flowers/test/102/image_08030.jpg
102
blackberry lily
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x705 at 0x7FBB2CC0CC88>


KeyError: 258

TypeError: crop() takes from 1 to 2 positional arguments but 5 were given

## Reminder for Workspace users
If your network becomes very large when saved as a checkpoint, there might be issues with saving backups in your workspace. You should reduce the size of your hidden layers and train again. 
    
We strongly encourage you to delete these large interim files and directories before navigating to another page or closing the browser tab.

In [ ]:
# TODO remove .pth files or move it to a temporary `~/opt` directory in this Workspace

In [ ]:
!tar cvfz project.tar.gz *

Image Classifier Project-zh.ipynb
Image Classifier Project.ipynb
LICENSE
README.md
assets/
assets/Flowers.png
assets/inference_example.png
cat_to_name.json
flowers
flowers_temp/
flowers_temp/train/
flowers_temp/train/25/
flowers_temp/train/25/image_06605.jpg
flowers_temp/train/25/image_06585.jpg
flowers_temp/train/25/image_06609.jpg
flowers_temp/train/25/image_06591.jpg
flowers_temp/train/25/image_06606.jpg
flowers_temp/train/25/image_06581.jpg
flowers_temp/train/25/image_06592.jpg
flowers_temp/train/25/image_06607.jpg
flowers_temp/train/25/image_06610.jpg
flowers_temp/train/25/image_06589.jpg
flowers_temp/train/25/image_06600.jpg
flowers_temp/train/25/image_06579.jpg
flowers_temp/train/25/image_06578.jpg
flowers_temp/train/25/image_06590.jpg
flowers_temp/train/25/image_06574.jpg
flowers_temp/train/25/image_06586.jpg
flowers_temp/train/25/image_06576.jpg
flowers_temp/train/25/image_06602.jpg
flowers_temp/train/25/image_06594.jpg
flowers_temp/train/25/image_06571.jpg
flowers_temp/train/

flowers_temp/train/95/image_07586.jpg
flowers_temp/train/95/image_07535.jpg
flowers_temp/train/95/image_07496.jpg
flowers_temp/train/95/image_07530.jpg
flowers_temp/train/95/image_07562.jpg
flowers_temp/train/95/image_07476.jpg
flowers_temp/train/95/image_07503.jpg
flowers_temp/train/95/image_07474.jpg
flowers_temp/train/95/image_07569.jpg
flowers_temp/train/95/image_07480.jpg
flowers_temp/train/95/image_07522.jpg
flowers_temp/train/95/image_07544.jpg
flowers_temp/train/95/image_07539.jpg
flowers_temp/train/95/image_07495.jpg
flowers_temp/train/95/image_07543.jpg
flowers_temp/train/95/image_07554.jpg
flowers_temp/train/95/image_07516.jpg
flowers_temp/train/95/image_07492.jpg
flowers_temp/train/95/image_07571.jpg
flowers_temp/train/95/image_07568.jpg
flowers_temp/train/95/image_07515.jpg
flowers_temp/train/95/image_07501.jpg
flowers_temp/train/95/image_07545.jpg
flowers_temp/train/95/image_07537.jpg
flowers_temp/train/95/image_07589.jpg
flowers_temp/train/95/image_07592.jpg
flowers_temp

flowers_temp/train/35/image_06973.jpg
flowers_temp/train/35/image_06969.jpg
flowers_temp/train/35/image_07000.jpg
flowers_temp/train/35/image_06985.jpg
flowers_temp/train/35/image_07003.jpg
flowers_temp/train/35/image_06988.jpg
flowers_temp/train/35/image_06993.jpg
flowers_temp/train/35/image_06987.jpg
flowers_temp/train/35/image_06998.jpg
flowers_temp/train/35/image_08087.jpg
flowers_temp/train/35/image_06974.jpg
flowers_temp/train/35/image_06976.jpg
flowers_temp/train/35/image_06999.jpg
flowers_temp/train/35/image_06980.jpg
flowers_temp/train/35/image_08086.jpg
flowers_temp/train/35/image_06979.jpg
flowers_temp/train/35/image_06982.jpg
flowers_temp/train/35/image_06990.jpg
flowers_temp/train/35/image_06995.jpg
flowers_temp/train/35/image_06971.jpg
flowers_temp/train/35/image_06997.jpg
flowers_temp/train/35/image_06972.jpg
flowers_temp/train/35/image_06996.jpg
flowers_temp/train/35/image_06970.jpg
flowers_temp/train/35/image_07002.jpg
flowers_temp/train/35/image_06989.jpg
flowers_temp

flowers_temp/train/73/image_00375.jpg
flowers_temp/train/73/image_00355.jpg
flowers_temp/train/73/image_00325.jpg
flowers_temp/train/73/image_00305.jpg
flowers_temp/train/73/image_00331.jpg
flowers_temp/train/73/image_00266.jpg
flowers_temp/train/73/image_00273.jpg
flowers_temp/train/73/image_00376.jpg
flowers_temp/train/73/image_00347.jpg
flowers_temp/train/73/image_00262.jpg
flowers_temp/train/73/image_00317.jpg
flowers_temp/train/73/image_00326.jpg
flowers_temp/train/73/image_00252.jpg
flowers_temp/train/73/image_00407.jpg
flowers_temp/train/73/image_00272.jpg
flowers_temp/train/73/image_00312.jpg
flowers_temp/train/73/image_00318.jpg
flowers_temp/train/73/image_00344.jpg
flowers_temp/train/73/image_00297.jpg
flowers_temp/train/73/image_00361.jpg
flowers_temp/train/73/image_00444.jpg
flowers_temp/train/73/image_00306.jpg
flowers_temp/train/73/image_00408.jpg
flowers_temp/train/73/image_00256.jpg
flowers_temp/train/73/image_00339.jpg
flowers_temp/train/73/image_00435.jpg
flowers_temp

flowers_temp/train/22/image_05374.jpg
flowers_temp/train/22/image_05365.jpg
flowers_temp/train/22/image_05385.jpg
flowers_temp/train/22/image_05386.jpg
flowers_temp/train/22/image_05363.jpg
flowers_temp/train/22/image_05359.jpg
flowers_temp/train/22/image_05371.jpg
flowers_temp/train/22/image_05387.jpg
flowers_temp/train/22/image_05350.jpg
flowers_temp/train/22/image_05394.jpg
flowers_temp/train/22/image_05373.jpg
flowers_temp/train/22/image_05351.jpg
flowers_temp/train/22/image_05396.jpg
flowers_temp/train/22/image_05390.jpg
flowers_temp/train/22/image_05342.jpg
flowers_temp/train/22/image_05375.jpg
flowers_temp/train/22/image_05393.jpg
flowers_temp/train/22/image_05377.jpg
flowers_temp/train/22/image_05344.jpg
flowers_temp/train/22/image_05395.jpg
flowers_temp/train/22/image_05356.jpg
flowers_temp/train/22/image_05347.jpg
flowers_temp/train/22/image_05345.jpg
flowers_temp/train/22/image_05346.jpg
flowers_temp/train/22/image_05348.jpg
flowers_temp/train/22/image_05369.jpg
flowers_temp

flowers_temp/train/78/image_01897.jpg
flowers_temp/train/78/image_01835.jpg
flowers_temp/train/78/image_01850.jpg
flowers_temp/train/78/image_01951.jpg
flowers_temp/train/78/image_01918.jpg
flowers_temp/train/78/image_01904.jpg
flowers_temp/train/78/image_01878.jpg
flowers_temp/train/78/image_01882.jpg
flowers_temp/train/78/image_01877.jpg
flowers_temp/train/78/image_01896.jpg
flowers_temp/train/78/image_01833.jpg
flowers_temp/train/78/image_01905.jpg
flowers_temp/train/78/image_01859.jpg
flowers_temp/train/78/image_01853.jpg
flowers_temp/train/78/image_01886.jpg
flowers_temp/train/78/image_01887.jpg
flowers_temp/train/78/image_01908.jpg
flowers_temp/train/78/image_01943.jpg
flowers_temp/train/78/image_01945.jpg
flowers_temp/train/78/image_01854.jpg
flowers_temp/train/78/image_01884.jpg
flowers_temp/train/78/image_01832.jpg
flowers_temp/train/78/image_01917.jpg
flowers_temp/train/78/image_01881.jpg
flowers_temp/train/78/image_01828.jpg
flowers_temp/train/78/image_01846.jpg
flowers_temp